# LSTM Training

In [323]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [324]:
import pandas as pd
import numpy as np
from numpy import *
import itertools
import os
import random

In [363]:
# read finished Data; case training
def readData(src, noConfFlag=True):
    dataseries = []
    for i in itertools.count():
        try:
            #print(i)
            dataseries.append(np.array(pd.read_json(src+ '/'+os.path.basename(src[:-4])+'_'+str(i).zfill(12)+'_keypoints.json').iloc[0,0]['pose_keypoints_2d']))
            if noConfFlag:
                dataseries[-1] = np.append(dataseries[-1][:2],np.delete(dataseries[-1][2:], slice(None, None, 3)))
        except ValueError:
            break
        except IndexError:
            if noConfFlag:
                dataseries.append(np.zeros(25*2))
            else:
                dataseries.append(np.zeros(25*3))
    return dataseries,i

In [364]:
labelNames=['rennen','gehen']
dataPaths=['data/output/rennen/','data/output/gehen/']
samplePaths=[]
data=[]
labels=[]
#collect paths to data
dirs0 = os.listdir(dataPaths[0])
for p in dirs0:
    samplePaths.append(dataPaths[0]+p+'0')
dirs1 = os.listdir(dataPaths[1])
for p in dirs1:
    samplePaths.append(dataPaths[1]+p+'1')

#shuffel data
random.shuffle(samplePaths)
#print(samplePaths)
minLength = 60
for smp in samplePaths:
    l = smp[-1:]
    smp = smp[:-1]
    td, i = readData(smp)
    if i<minLength:
        #minLength = i
        print(smp)
    else:
        data.append(td[10:60])
        if l == '0':
            labels.append(np.array([1,0]))
        elif l == '1':
            labels.append(np.array([0,1]))
        else:
            #labels.append(np.array([0,0]))
            print("can't load: ",smp)
    print('.', end="")
        
print("data loaded.")
print("min Length: ", minLength)

data/output/rennen/VID_20181212_160445.mp4
.................data/output/rennen/VID_20181212_160500.mp4
.data/output/rennen/VID_20181212_160510.mp4
....data/output/rennen/VID_20181212_160301.mp4
........................data/output/rennen/VID_20181212_160311.mp4
......data/output/rennen/VID_20181212_155945.mp4
...................data/output/rennen/VID_20181212_160734.mp4
.........data/output/rennen/VID_20181212_160440.mp4
.data/output/rennen/VID_20181212_155930.mp4
..data/output/gehen/VID_20181212_160300.mp4
...........data/output/rennen/VID_20181212_160337.mp4
....data/output/rennen/VID_20181212_160845.mp4
.......data/output/rennen/VID_20181212_155952.mp4
...............data/output/rennen/VID_20181212_160911.mp4
...data/output/rennen/VID_20181212_160316.mp4
.....data/output/rennen/VID_20181212_160959.mp4
.............data/output/rennen/VID_20181212_155701.mp4
...data/output/rennen/VID_20181212_160938.mp4
......data/output/rennen/VID_20181212_160003.mp4
....................data/output/re

In [365]:
#define constants

time_steps=50
#hidden LSTM units
num_units=128
# 25*3=75; 25*2=50
n_input=50
#learning rate for adam
learning_rate=0.1

n_classes=2
#size of batch
batch_size=7

In [366]:
#shape for LSTM
unscaledData = np.concatenate(data, axis=0).reshape((-1,time_steps*50))
l = np.concatenate(labels, axis=0).reshape((-1,2))

In [367]:
print(unscaledData.shape,l.shape)
print(unscaledData)
print(l)

(175, 2500) (175, 2)
[[  0.       0.       0.     ... 439.052  316.355  431.211 ]
 [  0.       0.       0.     ... 551.607  130.732  539.925 ]
 [ 87.9951 160.991   78.866  ...   0.     202.832  450.79  ]
 ...
 [  0.       0.       0.     ... 287.708  647.878  264.137 ]
 [  0.       0.     345.047  ... 397.288  441.629  398.571 ]
 [  0.       0.     249.287  ... 567.951   91.2048 556.213 ]]
[[0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0

In [368]:
#naiv standardize
#minval = np.min(unscaledData[np.nonzero(unscaledData)])
#maxval = np.max(unscaledData[np.nonzero(unscaledData)])
#print(minval,maxval)
#d = (unscaledData-minval)/(maxval-minval)
#minval = np.min(d[np.nonzero(d)])
#maxval = np.max(d[np.nonzero(d)])
#print(minval,maxval)

In [369]:
scaledData = []
unscaledData[unscaledData == 0] = np.NaN
for video in unscaledData:
    temp =[]
    for i in range(0,n_input):
        p=50
        frame = video[p*i:p*(i+1)]
        # replace 0 with x_avg or y_avg
        x_werte = frame[::2]
        y_werte = frame[1::2]
        x_avg = np.nanmean(x_werte)
        y_avg = np.nanmean(y_werte)
        x_werte[isnan(x_werte)] = x_avg
        y_werte[isnan(y_werte)] = y_avg
        # noramlize data with min max
        minval = np.min(frame)
        maxval = np.max(frame)
        temp.append((frame - minval)/(maxval-minval))
    scaledData.append(temp)

/home/martin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
/home/martin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':
/home/martin/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [370]:
d = np.nan_to_num(scaledData)
print(d)

[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.28360899 0.01642869 0.19803314 ... 1.         0.05366871 0.96246099]
  [0.33860669 0.02589716 0.25522812 ... 1.         0.08853086 0.94783053]
  [0.37839039 0.01053091 0.29536171 ... 0.90146671 0.4144289  0.87034236]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.13793849 0.29054006 0.1121794  ... 0.91303483 0.01011514 0.84689374]
  [0.15555551 0.2748734  0.11953177 ... 0.95324423 0.01571151 0.90655307]
  [0.16054593 0.26929308 0.13462425 ... 0.99484597 0.05701521 0.96881511]]

 [[0.15022296 0.37130481 0.12257376 ... 0.97629143 0.09870739 0.96828963]
  [0.1429587  0.361145

In [371]:
#split data in train and testset
testSize=batch_size 

train_data = d[testSize:,:]
train_labels = l[testSize:,:]
test_data = d[:testSize,:]
test_labels = l[:testSize,:]

In [372]:
test_data.shape, test_labels.shape

((7, 50, 50), (7, 2))

In [373]:
train_data.shape, train_labels.shape

((168, 50, 50), (168, 2))

In [374]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [375]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

In [376]:
#defining the network
lstm_layer=tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

In [377]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [378]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [379]:
#initialize variables
accList =[]
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=0
    while iter<trainSize//batch_size:
        print(iter)
        batch_x=train_data[batch_size*iter:batch_size*(iter+1)]
        
        batch_y=train_labels[batch_size*iter:batch_size*(iter+1)]
        #print(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter%1==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            accList.append(acc)
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1

    #calculating test accuracy
    test_data = test_data[:batch_size].reshape((-1, time_steps, n_input))
    test_label = test_labels[:batch_size]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

0
For iter  0
Accuracy  0.42857143
Loss  41.5687
__________________
1
For iter  1
Accuracy  0.5714286
Loss  24.412495
__________________
2
For iter  2
Accuracy  0.5714286
Loss  20.835052
__________________
3
For iter  3
Accuracy  0.42857143
Loss  20.22328
__________________
4
For iter  4
Accuracy  0.5714286
Loss  10.604745
__________________
5
For iter  5
Accuracy  0.71428573
Loss  4.1016574
__________________
6
For iter  6
Accuracy  0.71428573
Loss  1.7292207
__________________
7
For iter  7
Accuracy  0.2857143
Loss  2.3135521
__________________
8
For iter  8
Accuracy  0.14285715
Loss  5.276259
__________________
9
For iter  9
Accuracy  0.71428573
Loss  2.1127756
__________________
10
For iter  10
Accuracy  0.14285715
Loss  5.2864356
__________________
11
For iter  11
Accuracy  0.42857143
Loss  1.9790618
__________________
12
For iter  12
Accuracy  0.71428573
Loss  0.6100553
__________________
13
For iter  13
Accuracy  0.71428573
Loss  1.2483855
__________________
14
For iter  14
Accu

In [384]:
sum(accList)/len(accList)

0.5952381292978922

In [385]:
sess.close()

tf.reset_default_graph()

# Visualisation

In [29]:

show_data = d.reshape(-1,n_input)*500
show_label = [np.where(r==1)[0][0] for r in l]

In [30]:
#show_label

In [31]:
show_data.shape

(2250, 50)

In [32]:
testing = show_data[1]
testing[1]

322.6544923518044

In [33]:
pairs = [
        (1,8),(1,2),(1,5),(2,3),(3,4),   
        (5,6),(6,7),(8,9),(9,10),(10,11),
        (8,12),(12,13),(13,14),(1,0),(0,15), 
        (15,17),(0,16),(16,18),
        (14,19),(19,20),(14,21),(11,22),(22,23),
        (11,24)
        ]

In [34]:
pairs[0][1]

8

In [38]:
import pygame
import sys, traceback

# Initialize Pygame.
pygame.init()
# Set size of pygame window.
screen=pygame.display.set_mode((900,750))
# Create empty pygame surface.
background = pygame.Surface(screen.get_size())
# Fill the background white color.
background.fill((255, 255, 255))
# Convert Surface object to make blitting faster.
background = background.convert()
# Copy background to screen (position (0, 0) is upper left corner).
screen.blit(background, (0,0))
# Create Pygame clock object.
clock = pygame.time.Clock()

mainloop = True
# Desired framerate in frames per second. Try out other values.
FPS = 30
# How many seconds the "game" is played.
playtime = 0.0

count = 0.0

speed = 1

try:
    
    while mainloop:
        # Do not go faster than this framerate.
        milliseconds = clock.tick(FPS*speed)
        playtime += milliseconds / 1000.0

        for event in pygame.event.get():
            # User presses QUIT-button.
            if event.type == pygame.QUIT:
                mainloop = False
            elif event.type == pygame.KEYDOWN:
                # User presses ESCAPE-Key
                if event.key == pygame.K_ESCAPE:
                    mainloop = False

        # Print framerate and playtime in titlebar.
        currentSample=int(count/n_input)%len(show_label)
        text = "FPS: {0:.2f}   Playtime: {1:.2f}  NR: {2:d}  Label: ".format(clock.get_fps(), playtime, currentSample+1)
        text += labelNames[show_label[currentSample]]
        pygame.display.set_caption(text)
        background.fill((255, 255, 255))
        f = show_data[int(count)%(show_data.shape[0])]
        
        for p in pairs:
            x1=int(f[p[0]*2])
            y1=int(f[p[0]*2+1])
            x2=int(f[p[1]*2])
            y2=int(f[p[1]*2+1])
            #if not (x1 == 0 or x2 == 0 or y1 == 0 or y2 == 0):
                #pygame.draw.line(background, (0, 0, 0), [x1,y1], [x2,y2], 1)
        for i in range(0,f.shape[0],2):#2 for no conf
            #confColor = 255 - int(255*f[i+2])%255
            #pygame.draw.rect(background, (confColor, confColor, confColor), (int(f[i]),int(f[i+1]),4,4)) #bw
            #pygame.draw.rect(background, (confColor, 0, 255-confColor), (int(f[i]),int(f[i+1]),4,4)) #rb
            if i!=25:
                pygame.draw.rect(background, (0, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
            else:
                pygame.draw.rect(background, (255, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
        #screen.blit(background, (300-f[8*2],300-f[8*2+1]))#2 for no conf
        screen.blit(background, (0,0))
        #Update Pygame display.
        pygame.display.update()

        count += 1
except TypeError:
    print("Type error: ", count," ", i)
    traceback.print_exc(file=sys.stdout)
except:
    traceback.print_exc(file=sys.stdout)
else:
    pygame.quit()
# Finish Pygame.
pygame.quit()

# At the very last:
print("This game was played for {0:.2f} seconds".format(playtime))

This game was played for 20.63 seconds


In [ ]:
pygame.display.list_modes()

# Live Data-read

In [ ]:
import sys
import time
import logging
from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler
from watchdog.events import PatternMatchingEventHandler  

In [ ]:
class MyHandler(PatternMatchingEventHandler):
    patterns = ["*.json"]
    data = []

    def process(self, event):
        """
        event.event_type 
            'modified' | 'created' | 'moved' | 'deleted'
        event.is_directory
            True | False
        event.src_path
            path/to/observed/file
        """
        # the file will be processed there
        try:
            #print(i)
            f = pd.read_json(event.src_path).iloc[0,0]['pose_keypoints_2d']
            self.data.append(f)
            print(f)
        except ValueError:
            print("ERROR reading: ",event.src_path, event.event_type)


    def on_created(self, event):
        self.process(event)
        
observer = Observer()
observer.schedule(MyHandler(), '.')
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()